In [1]:
# importacion general de librerias y de visualizacion (matplotlib y seaborn)
import pandas as pd
import scipy 
import numpy as np
import random as rd
import nltk
import pickle
import sklearn
from sklearn.neighbors import NearestNeighbors
from collections import Counter
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import date
from datetime import datetime
from tqdm import tqdm

%matplotlib inline
nltk.download('stopwords')

plt.style.use('default') # haciendo los graficos un poco mas bonitos en matplotlib
#plt.rcParams['figure.figsize'] = (20, 10)

sns.set(style="whitegrid") # seteando tipo de grid en seaborn

pd.options.display.float_format = '{:20,.2f}'.format # suprimimos la notacion cientifica en los outputs

import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\igna-\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
def timer(start_time=None):
    if not start_time:
        start_time = datetime.now()
        return start_time
    elif start_time:
        thour, temp_sec = divmod((datetime.now() - start_time).total_seconds(), 3600)
        tmin, tsec = divmod(temp_sec, 60)
        print('\n Time taken: %i hours %i minutes and %s seconds.' % (thour, tmin, round(tsec, 2)))

# Parte 3

In [4]:
identity = pd.read_csv('train_identity.csv')
transaction = pd.read_csv('train_transaction.csv')

In [5]:
mergeado = identity.merge(transaction, left_on='TransactionID', right_on='TransactionID',
          how = 'right')

In [3]:
import xgboost as xgb
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, ConfusionMatrixDisplay, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
from sklearn.feature_extraction.text import CountVectorizer

Spliteo el 20% con el delta tiempo mayor

In [6]:
test_lenght = int(144233*0.2)
mergeado.sort_values(by = "TransactionDT")
train_lenght = (144233- test_lenght)
train = mergeado.iloc[0:train_lenght ,:]
test = mergeado.iloc[(144233- test_lenght):144233,:]

Encodeo el train set y el validation set

Mean encoding en id_30 y OHE en las demas categoricas

In [7]:
train["id_30"].fillna("Desconocido",inplace =True)
mean_encoder_id_30 = train.groupby(['id_30'])['isFraud'].mean().to_dict()

In [8]:
train["id_31"].fillna("Desconocido",inplace =True)
vec = CountVectorizer().fit(train["id_31"])
bag_of_words = vec.transform(train["id_31"])
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
filtro31 = list(filter(lambda c: c[0].isalpha() and c[1] > 2000, words_freq))


In [9]:
train["DeviceInfo"].fillna("Desconocido",inplace =True)
vec = CountVectorizer().fit(train["DeviceInfo"])
bag_of_words = vec.transform(train["DeviceInfo"])
sum_words = bag_of_words.sum(axis=0) 
words_freq = [(word, sum_words[0, idx]) for word, idx in vec.vocabulary_.items()]
words_freq =sorted(words_freq, key = lambda x: x[1], reverse=True)
filtroDI = list(filter(lambda c: c[0].isalpha() and c[1] > 2000, words_freq))


In [10]:
def selector(x, vec):
  for item in vec:
    if x.find(item[0]):
      return item[0]
    else:
      return 'other'

In [11]:
train['id_30'] =  train['id_30'].map(mean_encoder_id_30)
train['id_31'] = train['id_31'].map(lambda x: selector(x, filtro31))
train['DeviceInfo'] = train['DeviceInfo'].map(lambda x: selector(x, filtroDI))

X_cat = ['ProductCD','card4','id_34','id_15','id_23','id_31','DeviceType','DeviceInfo' ,'card6','P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9']

cont_df = train.drop(X_cat,axis =1)
cont_df['id_12'] = cont_df['id_12'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_13'] = cont_df['id_13'].astype(float).fillna(cont_df['id_13'].mean()) 
cont_df['id_16'] = cont_df['id_16'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_27'] = cont_df['id_27'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_28'] = cont_df['id_28'].replace('Found',1).replace('New',0).astype(float)
cont_df['id_29'] = cont_df['id_29'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_13'] = cont_df['id_13'].astype(float).fillna(cont_df['id_13'].mean()) 
cont_df['id_33'] = cont_df['id_33'].astype(str).map(lambda x : 'nan' if x.find('nan') != -1 else
                                               int(x.split('x')[0])*int(x.split('x')[1])).astype(float)
media = cont_df['id_33'].mean()
cont_df['id_33'] = cont_df['id_33'] /media
cont_df['id_35'] = cont_df['id_35'].replace('T',1).replace('F',0).astype(float)
cont_df['id_36'] = cont_df['id_36'].replace('T',1).replace('F',0).astype(float)
cont_df['id_37'] = cont_df['id_37'].replace('T',1).replace('F',0).astype(float)
cont_df['id_38'] = cont_df['id_38'].replace('T',1).replace('F',0).astype(float)

cont_df.apply(lambda x: x.fillna(x.mean(),inplace = True)) 
cont_df.apply(lambda x: x.fillna(0,inplace = True)) 
cat_df = train[X_cat].fillna('Others')

In [12]:
encoders = []
for columna in X_cat:    
    ohe = OneHotEncoder(handle_unknown = 'ignore')  # drop='first'
    encoded_data = (ohe.fit_transform(cat_df[[columna]].astype(str)).todense().astype(int))
    encoders.append(ohe)
    encoded_data = pd.DataFrame(encoded_data, index = cont_df.index ).add_prefix(columna[0:8])
    cont_df = pd.concat([cont_df, encoded_data], axis=1)
train = cont_df

In [13]:
test["id_30"].fillna("Desconocido",inplace =True)
test["id_31"].fillna("Desconocido",inplace =True)
test["DeviceInfo"].fillna("Desconocido",inplace =True)
                          
test['id_30'] =  test['id_30'].map(mean_encoder_id_30)
test['id_31'] = test['id_31'].map(lambda x: selector(x, filtro31))
test['DeviceInfo'] = test['DeviceInfo'].map(lambda x: selector(x, filtroDI))

X_cat = ['ProductCD','card4','id_34','id_15','id_23','id_31','DeviceType','DeviceInfo' ,'card6','P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9']

cont_df = test.drop(X_cat,axis =1)
cont_df['id_12'] = cont_df['id_12'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_13'] = cont_df['id_13'].astype(float).fillna(cont_df['id_13'].mean()) 
cont_df['id_16'] = cont_df['id_16'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_27'] = cont_df['id_27'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_28'] = cont_df['id_28'].replace('Found',1).replace('New',0).astype(float)
cont_df['id_29'] = cont_df['id_29'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_13'] = cont_df['id_13'].astype(float).fillna(cont_df['id_13'].mean()) 
cont_df['id_33'] = cont_df['id_33'].astype(str).map(lambda x : 'nan' if x.find('nan') != -1 else
                                               int(x.split('x')[0])*int(x.split('x')[1])).astype(float)
media = cont_df['id_33'].mean()
cont_df['id_33'] = cont_df['id_33'] /media
cont_df['id_35'] = cont_df['id_35'].replace('T',1).replace('F',0).astype(float)
cont_df['id_36'] = cont_df['id_36'].replace('T',1).replace('F',0).astype(float)
cont_df['id_37'] = cont_df['id_37'].replace('T',1).replace('F',0).astype(float)
cont_df['id_38'] = cont_df['id_38'].replace('T',1).replace('F',0).astype(float)

cont_df.apply(lambda x: x.fillna(x.mean(),inplace = True)) 
cont_df.apply(lambda x: x.fillna(0,inplace = True)) 
cat_df = test[X_cat].fillna('Others')

In [14]:
cont = 0 
for columna in X_cat:  
    encoded_data = (encoders[cont].transform(cat_df[[columna]].astype(str)).todense().astype(int))
    encoded_data = pd.DataFrame(encoded_data, index = cont_df.index ).add_prefix(columna[0:8])
    cont_df = pd.concat([cont_df, encoded_data], axis=1)
    cont += 1
test = cont_df

In [15]:
X_train = train.drop(['isFraud'], axis=1)
y_train = train.loc[:, 'isFraud']

In [16]:
X_test= test.drop(['isFraud'], axis=1)
y_test = test.loc[:, 'isFraud']

Entreno un XGBoost buscando hiperparametros

In [19]:
from xgboost import XGBClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

In [20]:
from hyperopt.pyll.base import scope
from hyperopt.pyll.stochastic import sample
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe


space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 0,9),
        'reg_alpha' : hp.quniform('reg_alpha', 0,15,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'n_estimators': hp.quniform('n_estimators', 150,301,10),
        'seed': 0
    }

In [21]:
def objective(space):
    clf=xgb.XGBClassifier(
                    n_estimators = int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']))
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="auc",
            early_stopping_rounds=10,verbose=False)
    
    accuracy = roc_auc_score(y_test, clf.predict_proba(X_test)[:,1])
    print ("SCORE:", accuracy)
    return {'loss': -accuracy, 'status': STATUS_OK }

In [22]:
trials = Trials()
rstate = np.random.default_rng(int(2020))

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        rstate = rstate,
                        trials = trials
                        )

SCORE:                                                                                                                 
0.8994205999438308                                                                                                     
SCORE:                                                                                                                 
0.8958270171443978                                                                                                     
SCORE:                                                                                                                 
0.8959141593123197                                                                                                     
SCORE:                                                                                                                 
0.8933430665783154                                                                                                     
SCORE:                                  

In [23]:
best_hyperparams

{'gamma': 4.546109710752271,
 'max_depth': 15.0,
 'n_estimators': 210.0,
 'reg_alpha': 1.0,
 'reg_lambda': 0.4931803761169766}

In [24]:
model = XGBClassifier(gamma = 3.807233425619809,
                      max_depth= 8,
                      n_estimators= 220,
                      reg_alpha= 5,
                      reg_lambda= 0.06118423963716291, 
                      random_state = 1)
model.fit(X_train,y_train)

[13:04:41] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.5.1/src/learner.cc:1115: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=3.807233425619809, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=8, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=220, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=1,
              reg_alpha=5, reg_lambda=0.06118423963716291, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [25]:
print(classification_report(y_train, model.predict(X_train)))

              precision    recall  f1-score   support

           0       0.99      1.00      1.00    112635
           1       0.98      0.61      0.75      2752

    accuracy                           0.99    115387
   macro avg       0.99      0.80      0.87    115387
weighted avg       0.99      0.99      0.99    115387



In [26]:
roc_auc_score(y_test, model.predict_proba(X_test)[:,1])

0.8910524377735394

In [27]:
with open('xgboostmodel', 'wb') as xgboostmodel:
    pickle.dump(model, xgboostmodel)
    xgboostmodel.close()

## Entreno un RandomForest

In [17]:
from sklearn.ensemble import RandomForestClassifier

In [29]:
from sklearn.model_selection import RandomizedSearchCV# Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 10, stop = 50, num = 10)]
max_features = ['auto', 'sqrt']
max_depth = [int(x) for x in np.linspace(10, 60, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]
random_grid = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [32]:
rf = RandomForestClassifier()
rf_random = RandomizedSearchCV(estimator = rf, param_distributions = random_grid, n_iter = 2, cv = 3, verbose=2, random_state=42, n_jobs = -1)

In [33]:
rf_random.fit(X_train,y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   3 out of   6 | elapsed:    9.3s remaining:    9.3s
[Parallel(n_jobs=-1)]: Done   6 out of   6 | elapsed:   27.5s finished


RandomizedSearchCV(cv=3, estimator=RandomForestClassifier(), n_iter=2,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 15, 20, 25, 30, 35,
                                                      40, 45, 50, 55, 60,
                                                      None],
                                        'max_features': ['auto', 'sqrt'],
                                        'min_samples_leaf': [1, 2, 4],
                                        'min_samples_split': [2, 5, 10],
                                        'n_estimators': [10, 14, 18, 23, 27, 32,
                                                         36, 41, 45, 50]},
                   random_state=42, verbose=2)

In [35]:
print(classification_report(y_train, rf_random.predict(X_train)))

              precision    recall  f1-score   support

           0       0.99      1.00      0.99    112635
           1       0.99      0.48      0.65      2752

    accuracy                           0.99    115387
   macro avg       0.99      0.74      0.82    115387
weighted avg       0.99      0.99      0.99    115387



In [36]:
roc_auc_score(y_test, rf_random.predict_proba(X_test)[:,1])

0.837922659019979

# Prueba competencia XGBoost

In [13]:
identity = pd.read_csv('test_identity.csv')
transaction = pd.read_csv('test_transaction.csv')

In [14]:
mergeado = identity.merge(transaction, left_on='TransactionID', right_on='TransactionID',

          how = 'right')

In [15]:
identity = None
transaction =None

In [16]:
for c in mergeado.columns:
  if c[:2] == 'id':
    mergeado.rename(columns = {c:c.replace('-','_')},inplace= True)

In [17]:
 test = mergeado

In [18]:
test["id_30"].fillna("Desconocido",inplace =True)
test["id_31"].fillna("Desconocido",inplace =True)
test["DeviceInfo"].fillna("Desconocido",inplace =True)

test['id_30'] = test['id_30'].map(mean_encoder_id_30)
test['id_31'] = test['id_31'].map(lambda x: selector(x, filtro31))
test['DeviceInfo'] = test['DeviceInfo'].map(lambda x: selector(x, filtroDI))

X_cat = ['ProductCD','card4','id_34','id_15','id_23','id_31','DeviceType','DeviceInfo' ,'card6','P_emaildomain','R_emaildomain','M1','M2','M3','M4','M5','M6','M7','M8','M9']

cont_df = test.drop(X_cat,axis =1)
cont_df['id_12'] = cont_df['id_12'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_13'] = cont_df['id_13'].astype(float).fillna(cont_df['id_13'].mean()) 
cont_df['id_16'] = cont_df['id_16'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_27'] = cont_df['id_27'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_28'] = cont_df['id_28'].replace('Found',1).replace('New',0).astype(float)
cont_df['id_29'] = cont_df['id_29'].replace('Found',1).replace('NotFound',0).astype(float)
cont_df['id_13'] = cont_df['id_13'].astype(float).fillna(cont_df['id_13'].mean()) 
cont_df['id_33'] = cont_df['id_33'].astype(str).map(lambda x : 'nan' if x.find('nan') != -1 else
                                               int(x.split('x')[0])*int(x.split('x')[1])).astype(float)
media = cont_df['id_33'].mean()
cont_df['id_33'] = cont_df['id_33'] /media
cont_df['id_35'] = cont_df['id_35'].replace('T',1).replace('F',0).astype(float)
cont_df['id_36'] = cont_df['id_36'].replace('T',1).replace('F',0).astype(float)
cont_df['id_37'] = cont_df['id_37'].replace('T',1).replace('F',0).astype(float)
cont_df['id_38'] = cont_df['id_38'].replace('T',1).replace('F',0).astype(float)

cont_df.apply(lambda x: x.fillna(x.mean(),inplace = True)) 
cont_df.apply(lambda x: x.fillna(0,inplace = True)) 
cat_df = test[X_cat].fillna('Others')

In [19]:
cont = 0 
for columna in X_cat:  
    encoded_data = (encoders[cont].transform(cat_df[[columna]].astype(str)).todense().astype(int))
    encoded_data = pd.DataFrame(encoded_data, index = cont_df.index ).add_prefix(columna[0:8])
    cont_df = pd.concat([cont_df, encoded_data], axis=1)
    cont += 1
test = cont_df

In [20]:
with open('xgboostmodel', 'rb') as file:
    model=pickle.load(file)
file.close()
model

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, enable_categorical=False,
              gamma=3.807233425619809, gpu_id=-1, importance_type=None,
              interaction_constraints='', learning_rate=0.300000012,
              max_delta_step=0, max_depth=8, min_child_weight=1, missing=nan,
              monotone_constraints='()', n_estimators=220, n_jobs=8,
              num_parallel_tree=1, predictor='auto', random_state=1,
              reg_alpha=5, reg_lambda=0.06118423963716291, scale_pos_weight=1,
              subsample=1, tree_method='exact', validate_parameters=1,
              verbosity=None)

In [24]:
sample = pd.read_csv('submit_XGBoost.csv')

In [25]:
sample['isFraud'] = 1 - model.predict_proba(test)

In [26]:
sample.to_csv('submit_XGBoost.csv', index = False)

Submit XGBoost<img src="xgsubmit.png" />